In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from numpy import average
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import statistics

In [2]:
wind = gpd.read_file('../data/USWTDB/uswtdb_v3_2_20201014.geojson')

In [3]:
wind.head()

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry
0,3005443.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.351089,35.091896,POINT (-118.35109 35.09190)
1,3072704.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364197,35.077644,POINT (-118.36420 35.07764)
2,3072695.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364410,35.077435,POINT (-118.36441 35.07744)
3,3072661.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.363762,35.077908,POINT (-118.36376 35.07791)
4,3005333.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.368690,35.075294,POINT (-118.36869 35.07529)


In [4]:
wind['rated_cap_mw'] = wind['t_cap']/1000

In [5]:
wind.rated_cap_mw.describe()

count    59571.000000
mean         1.828775
std          0.668494
min          0.050000
25%          1.500000
50%          1.800000
75%          2.300000
max          6.000000
Name: rated_cap_mw, dtype: float64

In [6]:
wind[wind['rated_cap_mw'] == 6.0]

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,rated_cap_mw
4884,3041854.0,RI,Block Island,2016.0,5.0,30.0,GE Wind,Haliade 150-6,6000.0,106.0,150.0,17671.46,181.1,3.0,3.0,-71.529114,41.110191,POINT (-71.52911 41.11019),6.0
4885,3041851.0,RI,Block Island,2016.0,5.0,30.0,GE Wind,Haliade 150-6,6000.0,106.0,150.0,17671.46,181.1,3.0,3.0,-71.507561,41.125713,POINT (-71.50756 41.12571),6.0
4886,3041852.0,RI,Block Island,2016.0,5.0,30.0,GE Wind,Haliade 150-6,6000.0,106.0,150.0,17671.46,181.1,3.0,3.0,-71.513947,41.119930,POINT (-71.51395 41.11993),6.0
4887,3041855.0,RI,Block Island,2016.0,5.0,30.0,GE Wind,Haliade 150-6,6000.0,106.0,150.0,17671.46,181.1,3.0,3.0,-71.537651,41.106396,POINT (-71.53765 41.10640),6.0
4888,3041853.0,RI,Block Island,2016.0,5.0,30.0,GE Wind,Haliade 150-6,6000.0,106.0,150.0,17671.46,181.1,3.0,3.0,-71.521164,41.114731,POINT (-71.52116 41.11473),6.0
12167,3105267.0,VA,Coastal Virginia Offshore Wind,2020.0,2.0,NaN,Siemens Gamesa Renewable Energy,SWT-6.0-154,6000.0,111.0,154.0,18626.50,188.0,1.0,3.0,-75.491638,36.886841,POINT (-75.49164 36.88684),6.0
12168,3105266.0,VA,Coastal Virginia Offshore Wind,2020.0,2.0,NaN,Siemens Gamesa Renewable Energy,SWT-6.0-154,6000.0,111.0,154.0,18626.50,188.0,1.0,3.0,-75.491577,36.886841,POINT (-75.49158 36.88684),6.0


In [7]:
statistics.mode(list(wind['rated_cap_mw']))

1.5

In [8]:
wind.t_hh.describe()

count    58629.000000
mean        79.186150
std         12.439689
min         19.000000
25%         80.000000
50%         80.000000
75%         80.000000
max        131.000000
Name: t_hh, dtype: float64

In [9]:
statistics.mode(list(wind['t_hh']))

80.0

In [10]:
wind.t_manu.describe()

count       59569
unique         68
top       GE Wind
freq        25858
Name: t_manu, dtype: object

In [11]:
wind.t_manu.value_counts().head(10)

GE Wind                            25858
Vestas                             13904
Siemens                             5144
Gamesa                              3076
Mitsubishi                          2791
Suzlon                              1316
Siemens Gamesa Renewable Energy     1212
Nordex                              1145
Acciona                              758
Clipper                              681
Name: t_manu, dtype: int64

In [12]:
wind.t_state.value_counts() #number of turbines in each state

TX    15895
CA     6833
IA     5723
OK     4397
KS     3507
IL     3141
CO     2741
MN     2698
ND     2121
OR     2064
WA     1772
NE     1385
IN     1371
WY     1366
MI     1337
NM     1259
NY     1142
SD     1060
MO      886
PA      751
MT      556
ID      541
OH      503
WI      452
WV      396
ME      392
UT      208
AZ      146
AK      142
HI      132
NC      105
MA       92
NH       84
MD       80
VT       73
NV       68
PR       63
RI       32
TN       18
NJ        6
CT        3
VA        2
FL        2
DE        1
GU        1
AR        1
Name: t_state, dtype: int64

In [13]:
wind.head(1)

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,rated_cap_mw
0,3005443.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.351089,35.091896,POINT (-118.35109 35.09190),0.095


In [14]:
wind_farms = wind.groupby('p_name').p_tnum.max().to_frame().reset_index()

In [15]:
wind_farms.sort_values(by='p_tnum', ascending=False).head()

,p_name,p_tnum
1581,unknown San Gorgonio Pass 1,731.0
1595,unknown Tehachapi Wind Resource Area 1,720.0
546,Historic Victory Garden (Wind Stream),549.0
787,Mesa Wind Farm,460.0
1191,Stateline Wind Project,454.0


In [16]:
wind_park_by_state = wind.groupby(['t_state', 'p_name', 'p_cap']).size().to_frame()

In [17]:
wind_park_by_state

0
t_state p_name                 p_cap      
AK      Bethel Wind Turbine    0.90      1
        Chevak                 0.40      4
        Delta Junction         0.10      1
                               0.90      2
        Emmonak 1              0.40      4
...                                    ...
WY      Rolling Hills          99.00    66
        Roundhouse             226.56   82
        Seven Mile Hill I & II 118.50   79
        Silver Sage            42.00    20
        Top of the World       200.20  110

[1484 rows x 1 columns]

In [18]:
wind_park_by_state = wind_park_by_state.reset_index()

In [19]:
wind_park_by_state.p_cap.sum()

113278.962

In [20]:
installed_wind = pd.read_excel('../data/installed_wind_capacity_by_state_Q3_2020.xls', skiprows=5, nrows=53)

In [21]:
installed_wind

,State,1999-12-31 00:00:00,2000-12-31 00:00:00,2001-12-31 00:00:00,2002-12-31 00:00:00,2003-12-31 00:00:00,2004-12-31 00:00:00,2005-12-31 00:00:00,2006-12-31 00:00:00,2007-12-31 00:00:00,...,2011-12-31 00:00:00,2012-12-31 00:00:00,2013-12-31 00:00:00,2014-12-31 00:00:00,2015-12-31 00:00:00,2016-12-31 00:00:00,2017-12-31 00:00:00,2018-12-31 00:00:00,2019-12-31 00:00:00,Q3 2020
0,Alabama,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0
1,Alaska,0.725,0.825,0.825,0.925,0.925,1.189,1.489,1.705,1.7050,...,11.0,59,62,62,62,62,62,63,64,64
2,Arizona,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,139.0,238,238,238,268,268,268,268,268,268
3,Arkansas,0.000,0.000,0.000,0.000,0.100,0.100,0.100,0.100,0.1000,...,0.1,0,0,0,0,0,0,0,0,0
4,California,1615.993,1615.993,1683.093,1823.067,2024.907,2094.977,2149.457,2376.107,2439.1070,...,3917.0,5542,5830,5917,6108,5662,5609,5885,5973,5871
5,Colorado,21.600,21.600,61.200,61.200,223.200,230.700,230.750,290.750,1066.7500,...,1805.0,2301,2332,2593,2992,3026,3104,3706,3762,4558
6,Connecticut,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,0.0,0,0,0,5,5,5,5,5,5
7,Delaware,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,2.0,2,2,2,2,2,2,2,2,2
8,District of Columbia,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0
9,Florida,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0


In [22]:
installed_wind.columns = installed_wind.columns.astype(str).str.split('-', 1).str[0]

In [47]:
installed_wind.to_csv('correct_state_install_cap.csv')

In [31]:
installed_wind = installed_wind.set_index('State')
iw2 = installed_wind.stack()
iw2

0
State                        
Alabama     1999          0.0
            2000          0.0
            2001          0.0
            2002          0.0
            2003          0.0
...                       ...
Grand Total 2016      82171.0
            2017      89077.0
            2018      96487.0
            2019     105583.0
            Q3 2020  111808.0

[1166 rows x 1 columns]

In [32]:
iw2 = iw2.to_frame().reset_index()
iw2

,State,level_1,0
0,Alabama,1999,0.0
1,Alabama,2000,0.0
2,Alabama,2001,0.0
3,Alabama,2002,0.0
4,Alabama,2003,0.0
...,...,...,...
1161,Grand Total,2016,82171.0
1162,Grand Total,2017,89077.0
1163,Grand Total,2018,96487.0
1164,Grand Total,2019,105583.0


In [33]:
iw2.columns = [['State', 'Year', 'Installed_Cap']]

In [45]:
iw2_states = iw2.iloc[0:1144]

In [46]:
iw2_states.to_csv('stacked_state_cap.csv')

In [25]:
installed_wind_2 = installed_wind.transpose()

In [26]:
us_installed_cap = installed_wind_2.iloc[:, 52:]
us_installed_cap = us_installed_cap.reset_index()
us_installed_cap.columns = [['Year', 'Grand_Total']]
us_installed_cap.to_csv('us_installed_cap2.csv')

In [27]:
us_installed_cap

,Year,Grand_Total
0,1999,2472.4780
1,2000,2539.3230
2,2001,4231.7730
3,2002,4687.3610
4,2003,6349.9420
5,2004,6723.1240
6,2005,9147.0640
7,2006,11574.5050
8,2007,16907.0495
9,2008,25410.0420


In [28]:
installed_wind = installed_wind.reset_index()

In [29]:
installed_wind.to_csv('state_installed_cap1.csv')

In [30]:
installed_wind

,State,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Q3 2020
0,Alabama,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0
1,Alaska,0.725,0.825,0.825,0.925,0.925,1.189,1.489,1.705,1.7050,...,11.0,59,62,62,62,62,62,63,64,64
2,Arizona,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,139.0,238,238,238,268,268,268,268,268,268
3,Arkansas,0.000,0.000,0.000,0.000,0.100,0.100,0.100,0.100,0.1000,...,0.1,0,0,0,0,0,0,0,0,0
4,California,1615.993,1615.993,1683.093,1823.067,2024.907,2094.977,2149.457,2376.107,2439.1070,...,3917.0,5542,5830,5917,6108,5662,5609,5885,5973,5871
5,Colorado,21.600,21.600,61.200,61.200,223.200,230.700,230.750,290.750,1066.7500,...,1805.0,2301,2332,2593,2992,3026,3104,3706,3762,4558
6,Connecticut,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,0.0,0,0,0,5,5,5,5,5,5
7,Delaware,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,2.0,2,2,2,2,2,2,2,2,2
8,District of Columbia,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0
9,Florida,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0
